<a href="https://colab.research.google.com/github/aadenisova/rnaseq_train/blob/main/HW_06_Scanpy%26Seurat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание

Сделайте несколько вспомогательных функций, которые вам позднее пригодятся:
1. Разбиение одного объекта AnnData на список других, соответствующих каждому индивидуальному батчу;

In [ ]:
# Загрузка R magic
%load_ext rpy2.ipython

# Установка нужных пакетов
!apt-get install libgmp3-dev libcairo2-dev libharfbuzz-dev libfribidi-dev libudunits2-dev
!gdown 1W0TuuQospxcasOX5cQdhT_Xd-k7-QAck
!tar xf library.tar.gz
!pip3 install scanpy[leiden] anndata2ri

# Необходимо, чтобы конкретно прогрузился matplotlib
import os
os.kill(os.getpid(), 9) 

In [1]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython
%R .libPaths("usr/local/lib/R/site-library/")

'/content/...,'/usr/loca...,'/usr/lib/...,'/usr/lib/...


In [2]:
import numpy as np
import scanpy as sc

adata = sc.read_10x_mtx("filtered_feature_bc_matrix")

In [3]:
adata.obs["batch"] = ["1"]*10000+ ["2"]*1996

#можно функцию
def get_batch(adata = adata):
   adata_by_batch = [adata[adata.obs["batch"]==i] for i in adata.obs["batch"].unique()]
   print('{} {}'.format(adata_by_batch[0].obs['batch'].unique(), adata_by_batch[1].obs['batch'].unique()))
   return adata_by_batch

adata_by_batch = get_batch(adata)

['1'] ['2']


2. Создание нового объекта со случайными N клетками и M генами из изначального;

In [4]:
def random_sample(adata_small = adata, N=10, M=10):
  n_cells = adata_small.shape[0]
  n_genes = adata_small.shape[1]

  cell_sample = np.random.choice(n_cells, size = N, replace=False)
  gene_sample = np.random.choice(n_genes, size = M, replace=False)
  adata_small = adata_small[cell_sample,:][:, gene_sample]

  return adata_small

random_sample(adata).obs, random_sample(adata).var

(                   batch
 GTTGAACTCTTCGGTC-1     1
 CTTCTAAGTGAGGATC-1     1
 ATCAGGTAGCGGGTAT-1     1
 TGCAGGCAGGTAAGAG-1     2
 TGTGATGCATGTAACC-1     2
 TGTTTGTGTCTACGAT-1     2
 AGTCACATCGCTACAA-1     1
 CAGGCCAGTCGCAACC-1     1
 TTGTGTTGTCGGTGAA-1     2
 TCTGTCGCACTATGTG-1     2,                    gene_ids    feature_types
 GALNT16     ENSG00000100626  Gene Expression
 ITCH-AS1    ENSG00000236388  Gene Expression
 FAM71C      ENSG00000180219  Gene Expression
 AGTR1       ENSG00000144891  Gene Expression
 ZNF281      ENSG00000162702  Gene Expression
 CD2BP2      ENSG00000169217  Gene Expression
 LINC02588   ENSG00000257842  Gene Expression
 ANP32D      ENSG00000139223  Gene Expression
 AC080128.2  ENSG00000285908  Gene Expression
 AL359962.2  ENSG00000273160  Gene Expression)

3. Сохранение вашего AnnData-объекта как Seurat-объекта в формате `.rds`.

In [6]:
%%R 
library(Seurat)

In [8]:
%%R -i adata

save_rds_from_adata <- function(adata, name = 'adata.to.seurat.rds') {
  adata.to.seurat <- as.Seurat(adata, data = NULL, counts = "X")
  saveRDS(adata.to.seurat, name)
  }

save_rds_from_adata(adata)